In [2]:
import pandas as pd
import re
import datetime

In [3]:
df = pd.read_csv('Uncleaned_DS_jobs.csv',index_col = 'index')
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
index,,,,,,,,,,,,,,
0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [4]:
# convert salary estimate to int (exp : 100-150)
df['Salary Estimate'] = df['Salary Estimate'].apply(lambda s : '-'.join(re.findall('(\d+)', s)))
df['Size'] = df['Size'].apply(lambda s : '-'.join(re.findall('(\d+)', s)))

In [5]:
# only company name , don't hay rating
df['Company Name'] = df['Company Name'].apply(lambda s: s.split('\n')[0])

In [6]:
# caculation age of company

x = datetime.datetime.now()
now_year = x.year
df['company_age'] = now_year - df['Founded']

In [7]:
del df['Founded']

In [8]:
df['company_age'] = df['company_age'].replace(2024,-1)

In [9]:
# create 2 columns : min and max salary
df['min_salary'] = df['Salary Estimate'].apply(lambda s : s.split('-')[0])
df['min_salary'] = df['min_salary'].astype(int)

df['max_salary'] = df['Salary Estimate'].apply(lambda s: s.split('-')[1])
df['max_salary'] = df['max_salary'].astype(int)

In [10]:
# create average salary
df['avg_salary'] = round( (df['min_salary'] + df['max_salary'])/2 , 4)

In [11]:
df['Location'].value_counts()

San Francisco, CA       69
New York, NY            50
Washington, DC          26
Boston, MA              24
Chicago, IL             22
                        ..
Portland, OR             1
San Ramon, CA            1
Sacramento, CA           1
Frederick, MD            1
Rancho Cucamonga, CA     1
Name: Location, Length: 207, dtype: int64

In [12]:
# 
df['job_state'] = df['Location'].apply(lambda s: s.split(', ')[-1])

In [13]:
dicts = { 'United States': 'US', 
        'New Jersey':'NJ',
         'California':'CA',
         'Texas':'TX',
         'Utah':'UT'}

In [14]:
df['job_state'] = df['job_state'].replace(dicts.keys(), dicts.values())

In [15]:
# if job state same location . this call same state
df['same_state'] = df.apply(lambda row :'1' if row['Location'] == row['Headquarters'] else '0', axis = 1)

In [16]:
# aws : amazon web services
lst = ['python','excel', 'hadoop', 'spark', 'aws', 'tableau', 'big data', 'machine learning', 'statistics']

In [17]:
# some infomation in job description
for skill in lst:
    df[skill] = df.apply(lambda row :'1' if skill in row["Job Description"].lower() else '0', axis = 1)

In [18]:
df['seniority'] = df.apply(lambda row : 'senior' if 'senior' in row['Job Description'].lower() else 'na', axis = 1)

In [19]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Type of ownership,Industry,...,python,excel,hadoop,spark,aws,tableau,big data,machine learning,statistics,seniority
index,,,,,,,,,,,,,,,,,,,,,
0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,"New York, NY","New York, NY",1001-5000,Nonprofit Organization,Insurance Carriers,...,0,0,0,0,1,0,0,1,1,senior
1,Data Scientist,137-171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,"Chantilly, VA","Herndon, VA",5001-10000,Company - Public,Research & Development,...,0,0,1,0,0,0,1,1,1,na
2,Data Scientist,137-171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,"Boston, MA","Boston, MA",1001-5000,Private Practice / Firm,Consulting,...,1,1,0,0,1,0,0,1,1,na
3,Data Scientist,137-171,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,"Newton, MA","Bad Ragaz, Switzerland",501-1000,Company - Public,Electrical & Electronic Manufacturing,...,1,1,0,0,1,0,0,1,0,na
4,Data Scientist,137-171,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,"New York, NY","New York, NY",51-200,Company - Private,Advertising & Marketing,...,1,1,0,0,0,0,0,1,1,na


In [22]:
#df.to_csv('clean.csv')